In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path

In [ ]:
class LuSNARDatasetPreparator:
    def __init__(self, dataset_root, output_dir):
        """
        Initialize the dataset preparator.
        
        Args:
            dataset_root (str): Path to the root of the LuSNAR dataset
            output_dir (str): Path to save the processed dataset
        """
        self.dataset_root = Path(dataset_root)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # Create directories for processed data
        self.processed_images_dir = self.output_dir / "images"
        self.processed_depth_dir = self.output_dir / "depth"
        self.processed_images_dir.mkdir(exist_ok=True)
        self.processed_depth_dir.mkdir(exist_ok=True)

    def get_timestamp_pairs(self):
        """Get pairs of timestamps from both cameras."""
        image1_timestamps = sorted([f.stem for f in (self.dataset_root / "image1" / "RGB").glob("*.png")])
        image2_timestamps = sorted([f.stem for f in (self.dataset_root / "image2" / "RGB").glob("*.png")])
        
        # Find common timestamps
        common_timestamps = set(image1_timestamps).intersection(set(image2_timestamps))
        return sorted(list(common_timestamps))

    def process_image_pair(self, timestamp):
        """
        Process a pair of stereo images and their corresponding depth map.
        
        Args:
            timestamp (str): The timestamp for the image pair
        
        Returns:
            tuple: Processed RGB image and depth map
        """
        # Read stereo images
        img1_path = self.dataset_root / "image1" / "RGB" / f"{timestamp}.png"
        img2_path = self.dataset_root / "image2" / "RGB" / f"{timestamp}.png"
        depth_path = self.dataset_root / "image1" / "Depth" / f"{timestamp}.png"
        
        img1 = cv2.imread(str(img1_path))
        img2 = cv2.imread(str(img2_path))
        depth_map = cv2.imread(str(depth_path), cv2.IMREAD_UNCHANGED)
        
        if img1 is None or img2 is None or depth_map is None:
            raise ValueError(f"Failed to load images for timestamp {timestamp}")

        # Convert BGR to RGB
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        
        # Normalize depth map if needed
        # Note: Adjust this based on the actual depth map format in your dataset
        if depth_map.dtype != np.uint16:
            depth_map = cv2.normalize(depth_map, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        
        return img1, depth_map

    def save_processed_pair(self, timestamp, image, depth):
        """
        Save the processed image and depth map.
        
        Args:
            timestamp (str): The timestamp for the pair
            image (np.ndarray): The processed RGB image
            depth (np.ndarray): The processed depth map
        """
        # Save RGB image
        image_path = self.processed_images_dir / f"{timestamp}.png"
        cv2.imwrite(str(image_path), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        
        # Save depth map
        depth_path = self.processed_depth_dir / f"{timestamp}.png"
        cv2.imwrite(str(depth_path), depth)

    def prepare_dataset(self):
        """Prepare the entire dataset."""
        timestamps = self.get_timestamp_pairs()
        print(f"Found {len(timestamps)} image pairs")
        
        for timestamp in tqdm(timestamps, desc="Processing image pairs"):
            try:
                image, depth = self.process_image_pair(timestamp)
                self.save_processed_pair(timestamp, image, depth)
            except Exception as e:
                print(f"Error processing timestamp {timestamp}: {e}")
                continue

In [ ]:
dataset_root = "/path/to/lusnar/dataset"
output_dir = "/path/to/output/processed_dataset"

# Initialize and run the dataset preparation
preparator = LuSNARDatasetPreparator(dataset_root, output_dir)
preparator.prepare_dataset()

print("Dataset preparation completed!")